In [1]:
# read the text file "OK.population" and store in the list called population
population = list()

# open the text file for reading
filepath = "C:\\districting-data\\"
filename = "OK.population"
file = open( filepath + filename ,"r")

# while the current line is not empty, read in a new county population
line = file.readline()

while line != "":
    # split the line into two "words": 
    #    word[0]: the county's number
    #    word[1]: the county's population
    words = line.split() 
    county_number = words[0]
    county_population = int(words[1]) # cast the string as type int
    
    # append to population list
    population.append(county_population)
    
    # read next line
    line = file.readline() 

file.close()
print("population = ",population)

population =  [255755, 11943, 45837, 20081, 50976, 15034, 124098, 6193, 27469, 60580, 11154, 11629, 69967, 14003, 42391, 12769, 41848, 47472, 46987, 73085, 27576, 34273, 52431, 115541, 40069, 8878, 47557, 10957, 16577, 86905, 69442, 33151, 77350, 29600, 41487, 15840, 6239, 5925, 25482, 13488, 6472, 7992, 20252, 15205, 3685, 4527, 4810, 718633, 50384, 10536, 20640, 12191, 37492, 3647, 9446, 603403, 22119, 14182, 22683, 11561, 2922, 7527, 5642, 15029, 9423, 5636, 11572, 45048, 41259, 42416, 46562, 70990, 26446, 2475, 4151, 31848, 34506]


In [2]:
# Read the Oklahoma county graph. It is stored in the file "OK.graph", with edges listed like this:
# 0 23 
# 0 30 
# 0 47 
# 0 76 
# 1 5 
# 1 8 
# 1 23 
# 1 33 
# ...

# County-level graphs for all states can be found here:
# https://github.com/AustinLBuchanan/county-level-districting/tree/master/data

# NetworkX has a built-in function for reading this kind of file
import networkx as nx
filepath = 'C:\\districting-data\\'
filename = 'OK.graph'
G = nx.read_edgelist( filepath + filename, nodetype=int )

print("The Oklahoma (county) graph has",G.number_of_nodes(),"nodes and",G.number_of_edges(),"edges.")
print("The Oklahoma graph has nodes",G.nodes)
print("The Oklahoma graph has edges",G.edges)

The Oklahoma (county) graph has 77 nodes and 195 edges.
The Oklahoma graph has nodes [0, 23, 30, 47, 76, 1, 5, 8, 33, 46, 61, 2, 10, 13, 15, 37, 42, 57, 66, 3, 25, 44, 74, 4, 17, 29, 49, 55, 9, 16, 6, 7, 22, 41, 54, 67, 40, 11, 53, 56, 45, 59, 62, 70, 48, 12, 21, 24, 28, 32, 51, 38, 52, 14, 18, 58, 71, 19, 34, 68, 20, 26, 39, 27, 35, 64, 69, 63, 31, 43, 75, 36, 60, 72, 65, 50, 73]
The Oklahoma graph has edges [(0, 23), (0, 30), (0, 47), (0, 76), (23, 1), (23, 5), (23, 22), (23, 33), (23, 47), (23, 76), (30, 21), (30, 38), (30, 47), (30, 51), (30, 52), (30, 76), (47, 16), (47, 21), (76, 20), (76, 22), (76, 52), (1, 5), (1, 8), (1, 33), (1, 46), (1, 61), (5, 9), (5, 16), (5, 61), (8, 11), (8, 33), (8, 46), (8, 53), (8, 56), (33, 6), (33, 11), (33, 22), (33, 54), (46, 3), (46, 53), (46, 61), (46, 74), (61, 3), (61, 9), (61, 25), (61, 62), (2, 10), (2, 13), (2, 15), (2, 37), (2, 42), (2, 57), (2, 66), (10, 15), (10, 48), (10, 66), (13, 37), (13, 38), (13, 42), (13, 51), (13, 52), (15, 14),

In [3]:
# We are to solve the following task:
# input: a population vector, a graph G=(V,E), desired number of districts k, population lower and upper bounds L and U
# output: a partition of the nodes into k districts (not necessarily connected!) 
#            each with population in [L,U] to minimize the number of "cut edges" 
#
# An edge {i,j} from E is cut if its endpoints i and j are assigned to different districts.
#
# For example, consider the 4x4 grid graph: 
#
#         & - & - & - &
#         |   |   |   |
#         & - & - & - &
#         |   |   |   |
#         & - & - & - &
#         |   |   |   |
#         & - & - & - &
#
# Here are two ways to split it into 4 equal-size districts:
#
#         &   &   &   &                & - &   & - &
#         |   |   |   |                |   |   |   |
#         &   &   &   &                & - &   & - &
#         |   |   |   |                             
#         &   &   &   &                & - &   & - &
#         |   |   |   |                |   |   |   |
#         &   &   &   &                & - &   & - &
#
#          12 cut edges                 8 cut edges
#
# The plan with 8 cut edges looks more compact.
#

import gurobipy as gp
from gurobipy import GRB

In [4]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

import math
k = 5          # number of districts
L = math.ceil((1-deviation/2)*sum(population)/k)
U = math.floor((1+deviation/2)*sum(population)/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 746519 and U = 754021 and k = 5


In [5]:
# create model 
m = gp.Model()

# create variables
x = m.addVars(G.nodes, k, vtype=GRB.BINARY) # x[i,j] equals one when county i is assigned to district j
y = m.addVars(G.edges, vtype=GRB.BINARY)  # y[u,v] equals one when edge {u,v} is cut

Academic license - for non-commercial use only - expires 2021-04-22
Using license file C:\Users\buchanan\gurobi.lic


In [6]:
# objective is to minimize cut edges
m.setObjective( gp.quicksum( y[u,v] for u,v in G.edges ), GRB.MINIMIZE )

In [7]:
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum(x[i,j] for j in range(k)) == 1 for i in G.nodes)

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( population[i] * x[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( population[i] * x[i,j] for i in G.nodes) <= U for j in range(k) )

# add constraints saying that edge {i,j} is cut if i is assigned to district v but j is not.
m.addConstrs( x[i,v] - x[j,v] <= y[i,j] for i,j in G.edges for v in range(k))

{(0, 23, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 23, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 23, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 23, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 23, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 30, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 30, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 30, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 30, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 30, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 47, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 47, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 47, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 47, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 47, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 76, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 76, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 76, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 76, 3): <gurobi.Constr 

In [8]:
# solve IP model
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1062 rows, 580 columns and 4080 nonzeros
Model fingerprint: 0xea8b4b85
Variable types: 0 continuous, 580 integer (580 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]
Presolve time: 0.00s
Presolved: 1062 rows, 580 columns, 4080 nonzeros
Variable types: 0 continuous, 580 integer (580 binary)
Found heuristic solution: objective 105.0000000
Found heuristic solution: objective 96.0000000

Root relaxation: objective 0.000000e+00, 623 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  385   96.00000    0.00000   100%     -    0s
H    0     0                      93.000000

In [9]:
# print the number of cut edges
print("The number of cut edges is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in range(k)]
district_populations = [ sum(population[i] for i in district) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",districts[j])

The number of cut edges is 39.0
District 0 has population 753313 and contains counties [0, 30, 76, 6, 7, 22, 41, 67, 40, 52, 20, 26, 39, 35, 64]
District 1 has population 747384 and contains counties [47, 65, 50, 73]
District 2 has population 751820 and contains counties [4, 29, 49, 55]
District 3 has population 751308 and contains counties [2, 10, 13, 15, 37, 42, 57, 66, 48, 24, 51, 38, 14, 18, 58, 71, 19, 34, 68, 27, 69, 63, 31, 43, 75]
District 4 has population 747526 and contains counties [23, 1, 5, 8, 33, 46, 61, 3, 25, 44, 74, 17, 9, 16, 54, 11, 53, 56, 45, 59, 62, 70, 12, 21, 28, 32, 36, 60, 72]


In [10]:
# check if the districts are connected
for j in range(k):
    print("Is district",j,"connected?", nx.is_connected( G.subgraph( districts[j] ) ) )

Is district 0 connected? True
Is district 1 connected? False
Is district 2 connected? True
Is district 3 connected? True
Is district 4 connected? True
